<a href="https://colab.research.google.com/github/AmeerHamza-345/Q2/blob/main/Function_calling_config_Temp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U -q "google-generativeai>=0.7.2"

In [2]:
from google.colab import userdata
import google.generativeai as genai

genai.configure(api_key=userdata.get("GOOGLE_API_KEY"))

In [3]:
def enable_lights():
    """Turn on the lighting system."""
    print("LIGHTBOT: Lights enabled.")


def set_light_color(rgb_hex: str):
    """Set the light color. Lights must be enabled for this to work."""
    print(f"LIGHTBOT: Lights set to {rgb_hex}.")


def stop_lights():
    """Stop flashing lights."""
    print("LIGHTBOT: Lights turned off.")


light_controls = [enable_lights, set_light_color, stop_lights]
instruction = """You are a helpful lighting system bot. You can turn lights on and off,
                and you can set the color. Do not perform any other tasks."""

model = genai.GenerativeModel(
    "models/gemini-1.5-pro",
    tools=light_controls,
    system_instruction=instruction
)

chat = model.start_chat()

In [4]:
from google.generativeai.types import content_types
from collections.abc import Iterable


def tool_config_from_mode(mode: str, fns: Iterable[str] = ()):
    """Create a tool config with the specified function calling mode."""
    return content_types.to_tool_config(
        {"function_calling_config": {"mode": mode, "allowed_function_names": fns}}
    )

#**Text-only mode: NONE**

In [5]:
tool_config = tool_config_from_mode("none")

In [6]:
response = chat.send_message(
    "Hello light-bot, what can you do?", tool_config=tool_config
)
print(response.text)

I can turn lights on and off, and I can set the color of the lights.



#**Automatic mode: AUTO**

In [7]:
tool_config = tool_config_from_mode("auto")

response = chat.send_message("Light this place up!", tool_config=tool_config)

In [8]:
response.parts[0]

function_call {
  name: "enable_lights"
  args {
  }
}

#**Function-calling mode: ANY**

In [9]:
available_fns = ["set_light_color", "stop_lights"]

tool_config = tool_config_from_mode("any", available_fns)

response = chat.send_message("Make this place ORANGE!", tool_config=tool_config)

In [10]:
print(response.parts[0])

function_call {
  name: "set_light_color"
  args {
    fields {
      key: "rgb_hex"
      value {
        string_value: "FFA500"
      }
    }
  }
}



In [11]:
def set_light_color(rgb_hex: str):
    """Set the light color. Lights must be enabled for this to work."""
    print(f"LIGHTBOT: Lights set to {rgb_hex}.")

#**Automatic function calling**

In [12]:
# available_fns = ["enable_lights"]
available_fns = ["enable_lights", "set_light_color", "stop_lights"]
tool_config = tool_config_from_mode("any", available_fns)

In [15]:
auto_chat = model.start_chat(enable_automatic_function_calling=True)
res = auto_chat.send_message("It's awful dark in here - make it Green and then switch off lights", tool_config=tool_config)

LIGHTBOT: Lights enabled.
LIGHTBOT: Lights set to 00FF00.
LIGHTBOT: Lights turned off.


In [14]:
auto_chat = model.start_chat(enable_automatic_function_calling=True)
res = auto_chat.send_message("Turn off the lights", tool_config=tool_config)

LIGHTBOT: Lights turned off.


In [16]:
res = auto_chat.send_message("Turn off the lights", tool_config=tool_config)

LIGHTBOT: Lights turned off.


In [18]:
def set_light_values(brightness, color_temp):
    """Set the brightness and color temperature of a room light. (mock API).

    Args:
        brightness: Light level from 0 to 100. Zero is off and 100 is full brightness
        color_temp: Color temperature of the light fixture, which can be `daylight`, `cool` or `warm`.

    Returns:
        A dictionary containing the set brightness and color temperature.
    """
    return {
        "brightness": brightness,
        "colorTemperature": color_temp
    }

In [19]:
model = genai.GenerativeModel(model_name='gemini-1.5-flash',
                              tools=[set_light_values])

In [24]:
chat = model.start_chat(enable_automatic_function_calling=True)
response = chat.send_message('Dim the lights so the room feels cozy and warm.')
response.text

Lights are now set to 50%


"OK. I've dimmed the lights to 50% brightness.  Is there anything else?\n"

In [20]:
from typing import Literal

def set_light_values(brightness: int, color_temp: Literal["daylight", "cool", "warm"]):
    """Set the brightness and color temperature of a room light. (mock API).

    Args:
        brightness: Light level from 0 to 100. Zero is off and 100 is full brightness
        color_temp: Color temperature of the light fixture, which can be `daylight`, `cool` or `warm`.

    Returns:
        A dictionary containing the set brightness and color temperature.
    """
    return {
        "brightness": brightness,
        "colorTemperature": color_temp
    }

model = genai.GenerativeModel(model_name='gemini-1.5-flash',
                              tools=[set_light_values])

chat = model.start_chat(enable_automatic_function_calling=True)
response = chat.send_message('Dim the lights so the room feels cozy and warm.')
print(response.text)

OK. I've dimmed the lights to 30% brightness and set the color temperature to warm.  Is there anything else?



In [21]:
def power_disco_ball(power: bool) -> bool:
    """Powers the spinning disco ball."""
    print(f"Disco ball is {'spinning!' if power else 'stopped.'}")
    return True


def start_music(energetic: bool, loud: bool, bpm: int) -> str:
    """Play some music matching the specified parameters.

    Args:
      energetic: Whether the music is energetic or not.
      loud: Whether the music is loud or not.
      bpm: The beats per minute of the music.

    Returns: The name of the song being played.
    """
    print(f"Starting music! {energetic=} {loud=}, {bpm=}")
    return "Never gonna give you up."


def dim_lights(brightness: float) -> bool:
    """Dim the lights.

    Args:
      brightness: The brightness of the lights, 0.0 is off, 1.0 is full.
    """
    print(f"Lights are now set to {brightness:.0%}")
    return True

In [22]:
# Set the model up with tools.
house_fns = [power_disco_ball, start_music, dim_lights]

model = genai.GenerativeModel(model_name="gemini-1.5-flash", tools=house_fns)

# Call the API.
chat = model.start_chat()
response = chat.send_message("Turn this place into a party!")

# Print out each of the function calls requested from this single call.
for part in response.parts:
    if fn := part.function_call:
        args = ", ".join(f"{key}={val}" for key, val in fn.args.items())
        print(f"{fn.name}({args})")

power_disco_ball(power=True)
start_music(loud=True, energetic=True, bpm=120.0)
dim_lights(brightness=0.5)


In [23]:
model._tools.to_proto()

[function_declarations {
   name: "power_disco_ball"
   description: "Powers the spinning disco ball."
   parameters {
     type_: OBJECT
     properties {
       key: "power"
       value {
         type_: BOOLEAN
       }
     }
     required: "power"
   }
 }
 function_declarations {
   name: "start_music"
   description: "Play some music matching the specified parameters.\n\n    Args:\n      energetic: Whether the music is energetic or not.\n      loud: Whether the music is loud or not.\n      bpm: The beats per minute of the music.\n\n    Returns: The name of the song being played.\n    "
   parameters {
     type_: OBJECT
     properties {
       key: "loud"
       value {
         type_: BOOLEAN
       }
     }
     properties {
       key: "energetic"
       value {
         type_: BOOLEAN
       }
     }
     properties {
       key: "bpm"
       value {
         type_: INTEGER
       }
     }
     required: "energetic"
     required: "loud"
     required: "bpm"
   }
 }
 functi